## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-30-17-02-16 +0000,bbc,Luigi Mangione will not face death penalty if ...,https://www.bbc.com/news/articles/c78egdg3r0po...
1,2026-01-30-17-00-24 +0000,nypost,Hochul takes direct hit at Bruce Blakeman with...,https://nypost.com/2026/01/30/us-news/kathy-ho...
2,2026-01-30-17-00-14 +0000,nypost,Don Lemon bragged about ‘fearless’ journalism ...,https://nypost.com/2026/01/30/us-news/don-lemo...
3,2026-01-30-17-00-00 +0000,wsj,Protein With a Side of Protein: Restaurants Ch...,https://www.wsj.com/business/hospitality/prote...
4,2026-01-30-16-55-08 +0000,nyt,Panama Court Strikes Down Hong Kong Firm’s Can...,https://www.nytimes.com/2026/01/30/business/pa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2451/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
64,trump,72
17,ice,25
15,new,18
66,warsh,17
22,don,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
209,2026-01-30-04-50-00 +0000,wsj,President Trump’s advisers have been told that...,https://www.wsj.com/economy/central-banking/tr...,162
240,2026-01-30-01-53-00 +0000,wsj,President Trump and Senate Democrats said ther...,https://www.wsj.com/politics/policy/white-hous...,158
30,2026-01-30-16-20-00 +0000,wsj,"Kevin Warsh, a central-bank insider-turned-cri...",https://www.wsj.com/economy/central-banking/th...,142
195,2026-01-30-08-58-57 +0000,wsj,Trump Picks Kevin Warsh as Next Fed Chair; Dow...,https://www.wsj.com/livecoverage/stock-market-...,138
221,2026-01-30-02-56-08 +0000,nypost,Trump plans to name Kevin Warsh as next Fed ch...,https://nypost.com/2026/01/29/business/trump-p...,137


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
209,162,2026-01-30-04-50-00 +0000,wsj,President Trump’s advisers have been told that...,https://www.wsj.com/economy/central-banking/tr...
44,112,2026-01-30-15-56-46 +0000,bbc,Former CNN host Don Lemon arrested after anti-...,https://www.bbc.com/news/articles/c24gm3dz36po...
240,79,2026-01-30-01-53-00 +0000,wsj,President Trump and Senate Democrats said ther...,https://www.wsj.com/politics/policy/white-hous...
363,63,2026-01-29-18-45-03 +0000,nypost,"Starbucks CEO lays out long-term growth plan, ...",https://nypost.com/2026/01/29/business/starbuc...
137,53,2026-01-30-12-13-00 +0000,wsj,Kevin Warsh’s nomination to lead the Federal R...,https://www.wsj.com/economy/central-banking/wh...
270,46,2026-01-29-23-45-33 +0000,cbc,"Prime minister meets Heated Rivalry stars, pay...",https://www.cbc.ca/news/canada/livestory/heate...
302,44,2026-01-29-21-57-12 +0000,nypost,Saks Off 5th to close most stores following ba...,https://nypost.com/2026/01/29/business/saks-of...
256,43,2026-01-30-00-16-50 +0000,cbc,Trump threatens to tariff and 'decertify' Cana...,https://www.cbc.ca/news/politics/trump-tariffs...
164,42,2026-01-30-11-00-00 +0000,nypost,New $65K private school uses AI to teach stude...,https://nypost.com/2026/01/30/business/new-65k...
53,40,2026-01-30-15-27-50 +0000,nypost,Luigi Mangione will not face death penalty in ...,https://nypost.com/2026/01/30/us-news/luigi-ma...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
